# Object Classification and Detection in Aerial Images
This project demonstrates a complete pipeline for classifying and detecting objects in aerial images.

In [ ]:
# Data Preprocessing
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, RandomHorizontalFlip, RandomRotation
from torch.utils.data import random_split, DataLoader

# Define transforms for data augmentation and preprocessing
transform = Compose([
    Resize((224, 224)),
    RandomHorizontalFlip(),
    RandomRotation(15),
    ToTensor()
])

# Load the dataset
dataset_path = "data/aerial_images"  # Update with your dataset path
dataset = ImageFolder(root=dataset_path, transform=transform)

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


In [ ]:
# Model Building: Classification with ResNet-50
import torch.nn as nn
from torchvision.models import resnet50

# Load pre-trained ResNet-50 model and modify for classification
num_classes = len(dataset.classes)  # Adjust based on dataset
model = resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=0.001)


In [ ]:
# Training the Model
import torch

def train_model(model, dataloader, loss_fn, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            # Move data to GPU if available
            inputs, labels = inputs.cuda(), labels.cuda() if torch.cuda.is_available() else (inputs, labels)
            
            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader):.4f}")
    
# Train the model
train_model(model, train_loader, loss_fn, optimizer, num_epochs=10)


In [ ]:
# Deployment with FastAPI
from fastapi import FastAPI, File, UploadFile
from PIL import Image

app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = Image.open(file.file).convert("RGB")
    image = transform(image).unsqueeze(0)  # Preprocess the image
    model.eval()
    with torch.no_grad():
        pred = model(image)
    return {"Prediction": pred.argmax(1).item()}


In [ ]:
# Dockerfile for Deployment
# Use Python slim image
FROM python:3.8-slim

# Set working directory
WORKDIR /app

# Copy requirements and install dependencies
COPY requirements.txt .
RUN pip install -r requirements.txt

# Copy application code
COPY . .

# Expose API port and run FastAPI app
EXPOSE 8000
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
